In [3]:
# ==== Minimal, conflict-safe setup (Colab/Kaggle) ====
# Installs only what we need, without touching core environment packages.

%pip -q install --no-deps pyBKT==1.4.1
%pip -q install --no-deps lightgbm==4.3.0
%pip -q install --no-deps sentence-transformers==3.0.1
%pip -q install --no-deps faiss-cpu==1.8.0.post1
%pip -q install --no-deps jinja2==3.1.4

# Optional: install transformers/torch ONLY if they’re missing (keeps environment stable)
try:
    import torch, transformers  # noqa: F401
except Exception:
    %pip -q install --upgrade-strategy only-if-needed torch transformers

# ---- Sanity checks & device setup ----
import warnings, os
warnings.filterwarnings("ignore")

# Core imports
import pandas as pd, numpy as np
import lightgbm as lgb
from pyBKT.models import Model
from sentence_transformers import SentenceTransformer

# Device (GPU used by Sentence-Transformers if available)
try:
    import torch
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
except Exception:
    DEVICE = "cpu"

print("✅ Environment OK")
print("  - PyBKT:", Model.__module__)
print("  - LightGBM:", lgb.__version__)
print("  - Torch device:", DEVICE)

# Quick embedding model warm-up (downloads to cache)
_ = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2", device=DEVICE)
print("  - Sentence-Transformers ready")


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


2025-10-07 04:59:53.459650: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759813193.690432      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759813193.762044      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


✅ Environment OK
  - PyBKT: pyBKT.models.Model
  - LightGBM: 4.3.0
  - Torch device: cuda


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

  - Sentence-Transformers ready


In [5]:
# ==== Locate dataset, (optional) copy to /kaggle/working, and load ALL CSVs ====
from pathlib import Path
import shutil
import pandas as pd

# Files we expect
REQUIRED = {
    "students.csv",
    "courses.csv",
    "topics.csv",
    "assessments.csv",
    "engagement.csv",
    "feedback.csv",
    "course_summary_grades.csv",
}

# 1) Find DATA folder
DATA = Path(".")  # default: current dir (in case you've already copied files here)
def contains_required(dirpath: Path) -> bool:
    try:
        names = {p.name for p in dirpath.glob("*.csv")}
        return REQUIRED.issubset(names)
    except Exception:
        return False

if not contains_required(DATA):
    # Try Kaggle input mounts
    for d in Path("/kaggle/input").glob("*"):
        if contains_required(d):
            DATA = d
            break

print("Using DATA folder:", DATA)
print("Files in DATA:", sorted([p.name for p in DATA.glob('*.csv')]))
assert contains_required(DATA), "Could not find all required CSVs in DATA."

# 2) (Optional) Copy inputs to /kaggle/working so artifacts land together
COPY_TO_WORKING = False  # set True if you want local copies
if COPY_TO_WORKING:
    DST = Path("/kaggle/working")
    DST.mkdir(parents=True, exist_ok=True)
    for fname in REQUIRED:
        shutil.copy(DATA/fname, DST/fname)
    print("Copied to /kaggle/working")

# 3) Load ALL CSVs into dataframes
students  = pd.read_csv(DATA/"students.csv")
courses   = pd.read_csv(DATA/"courses.csv")
topics    = pd.read_csv(DATA/"topics.csv")
assess    = pd.read_csv(DATA/"assessments.csv", parse_dates=["timestamp"])
engage    = pd.read_csv(DATA/"engagement.csv")
feedback  = pd.read_csv(DATA/"feedback.csv")
grades    = pd.read_csv(DATA/"course_summary_grades.csv")

# 4) Quick sanity print
for name, df in [
    ("students", students),
    ("courses", courses),
    ("topics", topics),
    ("assessments", assess),
    ("engagement", engage),
    ("feedback", feedback),
    ("grades", grades),
]:
    print(f"{name:12s} -> shape={df.shape}, cols={list(df.columns)[:6]}{'...' if df.shape[1]>6 else ''}")


Using DATA folder: /kaggle/input/student-progress
Files in DATA: ['assessments.csv', 'course_summary_grades.csv', 'courses.csv', 'engagement.csv', 'feedback.csv', 'students.csv', 'topics.csv']
students     -> shape=(200, 4), cols=['student_id', 'name', 'batch_year', 'department']
courses      -> shape=(20, 5), cols=['course_code', 'course_title', 'semester_no', 'course_type', 'credit_hours']
topics       -> shape=(119, 3), cols=['course_code', 'topic_id', 'topic_name']
assessments  -> shape=(24000, 8), cols=['student_id', 'course_code', 'semester_no', 'assessment_type', 'topic_id', 'raw_score']...
engagement   -> shape=(4000, 8), cols=['student_id', 'course_code', 'semester_no', 'attendance_percentage', 'lms_logins', 'forum_posts']...
feedback     -> shape=(4000, 6), cols=['student_id', 'course_code', 'semester_no', 'interest_rating', 'perceived_difficulty', 'comments']
grades       -> shape=(4000, 7), cols=['student_id', 'course_code', 'semester_no', 'course_type', 'credit_hours', 'fi

In [6]:
# =========================================================
# 2) Feature Engineering + New Policy Labels
# Final% = 10% Att + 15% (quizzes+assign) + 25% Mid + 50% Final
# =========================================================
LETTER_BANDS = [("A+",80,101),("A",75,80),("A-",70,75),("B+",65,70),("B",60,65),("B-",55,60),
                ("C+",50,55),("C",45,50),("D",40,45),("F",0,40)]
def letter_from_pct(p):
    for band, lo, hi in LETTER_BANDS:
        if p>=lo and p<hi: return band
    return "F"

def add_course_meta(df):
    return df.merge(courses[["course_code","semester_no","course_type","credit_hours"]]
                    .drop_duplicates(), on=["course_code","semester_no"], how="left")

assess["norm"] = (assess["raw_score"]/assess["max_score"]).clip(0,1)

def build_assessment_wide(df):
    piv = df.pivot_table(index=["student_id","course_code","semester_no"],
                         columns="assessment_type", values="norm", aggfunc="mean").reset_index()
    for c in ["quiz1","quiz2","quiz3","assignment","mid","final"]:
        if c not in piv.columns: piv[c] = np.nan
    piv["CA_qa"] = piv[["quiz1","quiz2","quiz3","assignment"]].mean(axis=1, skipna=True).fillna(0)
    piv["Mid_norm"]   = piv["mid"].fillna(0)
    piv["Final_norm"] = piv["final"].fillna(0)
    return piv

wide = build_assessment_wide(assess)

# Attendance merge
att = engage.groupby(["student_id","course_code","semester_no"], as_index=False)["attendance_percentage"].mean()
wide = wide.merge(att, on=["student_id","course_code","semester_no"], how="left")
wide["Att"] = wide["attendance_percentage"].fillna(0)/100.0

# New Policy Final %
wide["final_percentage_new"] = (0.10*wide["Att"] + 0.15*wide["CA_qa"] +
                                0.25*wide["Mid_norm"] + 0.50*wide["Final_norm"]) * 100

wide["letter_grade_new"] = wide["final_percentage_new"].apply(letter_from_pct)

# Save a new policy summary (optional)
new_policy = wide[["student_id","course_code","semester_no","final_percentage_new","letter_grade_new"]].copy()
new_policy.to_csv("course_summary_grades_NEW_POLICY.csv", index=False)
print("Saved course_summary_grades_NEW_POLICY.csv", new_policy.shape)


Saved course_summary_grades_NEW_POLICY.csv (4000, 5)


In [7]:
# =========================================================
# 3) Supervised Learning Tables (Regression & Risk)
# =========================================================
def join_engagement(piv):
    agg_e = engage.groupby(["student_id","course_code","semester_no"], as_index=False).agg({
        "attendance_percentage":"mean",
        "lms_logins":"sum","forum_posts":"sum","chatbot_interactions":"sum",
        "on_time_submission_rate":"mean"
    })
    out = piv.merge(agg_e, on=["student_id","course_code","semester_no"], how="left")
    return add_course_meta(out)

def join_feedback(piv):
    fb = feedback.groupby(["student_id","course_code","semester_no"], as_index=False).agg({
        "interest_rating":"mean","perceived_difficulty":"mean"
    })
    return piv.merge(fb, on=["student_id","course_code","semester_no"], how="left")

def build_regression_table():
    X = build_assessment_wide(assess)
    X = join_engagement(X)
    X = join_feedback(X)
    y = new_policy.rename(columns={
        "final_percentage_new":"final_percentage",
        "letter_grade_new":"letter_grade"
    })
    df = X.merge(y, on=["student_id","course_code","semester_no"], how="inner")
    # prior semester average
    prior = (y.assign(prior_sem=lambda d: d["semester_no"]-1)
               .merge(y[["student_id","semester_no","final_percentage"]]
               .rename(columns={"semester_no":"prior_sem","final_percentage":"prior_pct"}),
               on=["student_id","prior_sem"], how="left"))
    prior_gpa = (prior.groupby(["student_id","semester_no"], as_index=False)["prior_pct"]
                      .mean().rename(columns={"prior_pct":"prior_sem_avg"}))
    df = df.merge(prior_gpa, on=["student_id","semester_no"], how="left")
    return df.fillna(0)

reg = build_regression_table()
reg.head(3)


,student_id,course_code,semester_no,assignment,final,mid,quiz1,quiz2,quiz3,CA_qa,...,forum_posts,chatbot_interactions,on_time_submission_rate,course_type,credit_hours,interest_rating,perceived_difficulty,final_percentage,letter_grade,prior_sem_avg
0,CSE240001,CSE 4301,3,0.75000,0.68333,0.77500,0.70000,0.70000,0.90000,0.76250,...,6,5,0.80000,Theory,3.00000,2.00000,3.00000,73.80517,A-,0.00000
1,CSE240001,CSE 4302,3,0.70000,0.78333,0.72500,0.80000,0.80000,0.80000,0.77500,...,2,3,0.96000,Lab,1.50000,2.00000,2.00000,77.26867,A,0.00000
2,CSE240001,CSE 4303,3,0.70000,0.71667,0.75000,0.80000,0.70000,0.80000,0.75000,...,3,8,0.77000,Theory,3.00000,2.00000,4.00000,74.64133,A-,0.00000


In [8]:
# =========================================================
# 3) Supervised Learning Tables (Regression & Risk)
# =========================================================
def join_engagement(piv):
    agg_e = engage.groupby(["student_id","course_code","semester_no"], as_index=False).agg({
        "attendance_percentage":"mean",
        "lms_logins":"sum","forum_posts":"sum","chatbot_interactions":"sum",
        "on_time_submission_rate":"mean"
    })
    out = piv.merge(agg_e, on=["student_id","course_code","semester_no"], how="left")
    return add_course_meta(out)

def join_feedback(piv):
    fb = feedback.groupby(["student_id","course_code","semester_no"], as_index=False).agg({
        "interest_rating":"mean","perceived_difficulty":"mean"
    })
    return piv.merge(fb, on=["student_id","course_code","semester_no"], how="left")

def build_regression_table():
    X = build_assessment_wide(assess)
    X = join_engagement(X)
    X = join_feedback(X)
    y = new_policy.rename(columns={
        "final_percentage_new":"final_percentage",
        "letter_grade_new":"letter_grade"
    })
    df = X.merge(y, on=["student_id","course_code","semester_no"], how="inner")
    # prior semester average
    prior = (y.assign(prior_sem=lambda d: d["semester_no"]-1)
               .merge(y[["student_id","semester_no","final_percentage"]]
               .rename(columns={"semester_no":"prior_sem","final_percentage":"prior_pct"}),
               on=["student_id","prior_sem"], how="left"))
    prior_gpa = (prior.groupby(["student_id","semester_no"], as_index=False)["prior_pct"]
                      .mean().rename(columns={"prior_pct":"prior_sem_avg"}))
    df = df.merge(prior_gpa, on=["student_id","semester_no"], how="left")
    return df.fillna(0)

reg = build_regression_table()
reg.head(3)


,student_id,course_code,semester_no,assignment,final,mid,quiz1,quiz2,quiz3,CA_qa,...,forum_posts,chatbot_interactions,on_time_submission_rate,course_type,credit_hours,interest_rating,perceived_difficulty,final_percentage,letter_grade,prior_sem_avg
0,CSE240001,CSE 4301,3,0.75000,0.68333,0.77500,0.70000,0.70000,0.90000,0.76250,...,6,5,0.80000,Theory,3.00000,2.00000,3.00000,73.80517,A-,0.00000
1,CSE240001,CSE 4302,3,0.70000,0.78333,0.72500,0.80000,0.80000,0.80000,0.77500,...,2,3,0.96000,Lab,1.50000,2.00000,2.00000,77.26867,A,0.00000
2,CSE240001,CSE 4303,3,0.70000,0.71667,0.75000,0.80000,0.70000,0.80000,0.75000,...,3,8,0.77000,Theory,3.00000,2.00000,4.00000,74.64133,A-,0.00000


In [10]:
# ==== Final-Grade Regressor (LightGBM) — fixed for no 'verbose' kw ====
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_absolute_error, mean_squared_error
import lightgbm as lgb
import numpy as np

FEATURES = [
    "quiz1","quiz2","quiz3","assignment","CA_qa","Mid_norm","Final_norm",
    "attendance_percentage","lms_logins","forum_posts","chatbot_interactions","on_time_submission_rate",
    "interest_rating","perceived_difficulty","credit_hours","prior_sem_avg"
]
CAT = ["course_type","course_code"]

# Ensure categorical dtypes for LightGBM
for c in CAT:
    reg[c] = reg[c].astype("category")

X = reg[FEATURES+CAT]
y = reg["final_percentage"]
groups = reg["semester_no"]

gkf = GroupKFold(n_splits=2)
oof, models = np.zeros(len(reg)), []

for tr, va in gkf.split(X, y, groups=groups):
    model = lgb.LGBMRegressor(
        n_estimators=1200,
        learning_rate=0.03,
        subsample=0.9,
        colsample_bytree=0.9,
        random_state=42
    )
    model.fit(
        X.iloc[tr], y.iloc[tr],
        eval_set=[(X.iloc[va], y.iloc[va])],
        eval_metric="l2",
        callbacks=[
            lgb.early_stopping(stopping_rounds=100),
            lgb.log_evaluation(period=50)  # prints every 50 iters; remove if you want silence
        ]
    )
    pred = model.predict(X.iloc[va])
    oof[va] = pred
    models.append(model)

print("Regressor MAE:", mean_absolute_error(y, oof))
print("Regressor RMSE:", mean_squared_error(y, oof, squared=False))

grade_model = models[0]


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000184 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 559
[LightGBM] [Info] Number of data points in the train set: 2000, number of used features: 17
[LightGBM] [Info] Start training from score 71.892533
Training until validation scores don't improve for 100 rounds
[50]	valid_0's l2: 5.6584
[100]	valid_0's l2: 0.536223
[150]	valid_0's l2: 0.174371
[200]	valid_0's l2: 0.137392
[250]	valid_0's l2: 0.129542
[300]	valid_0's l2: 0.125384
[350]	valid_0's l2: 0.121882
[400]	valid_0's l2: 0.120283
[450]	valid_0's l2: 0.118828
[500]	valid_0's l2: 0.117819
[550]	valid_0's l2: 0.116915
[600]	valid_0's l2: 0.115981
[650]	valid_0's l2: 0.115018
[700]	valid_0's l2: 0.114079
[750]	valid_0's l2: 0.113735
[800]	valid_0's l2: 0.112956
[850]	valid_0's l2: 0.112747
[900]	valid_0's l2: 0.112446
[950]	valid_0'

In [11]:
# =========================================================
# 5) Early Risk Classifier (after-mid checkpoint)
# =========================================================
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score

# Use features available after midterm (exclude final_norm)
AFTER_MID = [
    "quiz1","quiz2","quiz3","assignment","CA_qa","Mid_norm",
    "attendance_percentage","lms_logins","forum_posts","chatbot_interactions","on_time_submission_rate",
    "interest_rating","perceived_difficulty","credit_hours","course_type","course_code"
]
risk_df = reg.copy()
target_threshold = 60.0
risk_df["risk_label"] = (risk_df["final_percentage"] < target_threshold).astype(int)

Xr = risk_df[AFTER_MID]
Xr_enc = pd.get_dummies(Xr, columns=["course_type","course_code"], drop_first=True)
yr = risk_df["risk_label"]

risk_model = LogisticRegression(max_iter=500)
risk_model.fit(Xr_enc, yr)
prob = risk_model.predict_proba(Xr_enc)[:,1]
print("Risk ROC-AUC:", roc_auc_score(yr, prob))
print("Risk PR-AUC:", average_precision_score(yr, prob))


Risk ROC-AUC: 0.9782412810021123
Risk PR-AUC: 0.8599434930601237


In [15]:
# ==== Topic Mastery with BKT (robust to pyBKT variants) ====
# Produces P_mastery with columns: user_id, skill_name, p_mastery (latest per topic)

from pyBKT.models import Model
import pandas as pd
import numpy as np

# 1) Build per-attempt sequences
seq = assess.copy()
seq["correct"] = (seq["raw_score"] / seq["max_score"] >= 0.6).astype(int)
seq = (seq.merge(topics, on=["course_code","topic_id"], how="left")
          .dropna(subset=["topic_name"]))

seq_bkt = (seq.rename(columns={"student_id":"user_id", "topic_name":"skill_name"})
              [["user_id","skill_name","correct","timestamp"]]
              .sort_values(["user_id","skill_name","timestamp"])
              .reset_index(drop=True))
seq_bkt["user_id"]    = seq_bkt["user_id"].astype(str)
seq_bkt["skill_name"] = seq_bkt["skill_name"].astype(str)
seq_bkt["seq_idx"]    = seq_bkt.groupby(["user_id","skill_name"]).cumcount()

print(f"Unique skills: {seq_bkt['skill_name'].nunique()} | Rows: {len(seq_bkt)}")

# 2) Fit pyBKT across all skills
bkt = Model(seed=42, num_fits=5)
bkt.fit(data=seq_bkt[["user_id","skill_name","correct","timestamp"]])

# 3) Predict mastery per interaction
mastery = bkt.predict(data=seq_bkt[["user_id","skill_name","correct","timestamp"]]).reset_index(drop=True)

# Attach alignment columns
mastery["user_id"]    = seq_bkt["user_id"].values
mastery["skill_name"] = seq_bkt["skill_name"].values
mastery["seq_idx"]    = seq_bkt["seq_idx"].values

# 4) Extract p_mastery from whatever the model returned
#    - Preferred: 'posterior' (some builds)
#    - Common alt: 'state_predictions' (prob vector over states at this step)
#    - Fallback:    use 'correct_predictions' as a proxy (less ideal)

pcol = None
for cand in mastery.columns:
    if cand.lower().startswith("posterior"):
        pcol = cand
        break

if pcol is None and "state_predictions" in mastery.columns:
    def to_known_prob(v):
        # v is a list/array of state probabilities for this timestep
        a = np.array(v, dtype=float).ravel()
        # pyBKT often orders states as [unlearned, learned]; take index 1 if size==2.
        if a.size >= 2:
            return float(a[1])   # assume index 1 = "Known"
        # if a single value or unexpected shape, use max as a conservative proxy
        return float(a.max()) if a.size else np.nan
    mastery["p_mastery"] = mastery["state_predictions"].apply(to_known_prob)
else:
    if pcol is not None:
        mastery["p_mastery"] = mastery[pcol].astype(float)
    elif "correct_predictions" in mastery.columns:
        # Fallback proxy: probability of a correct response (not pure mastery, but usable)
        mastery["p_mastery"] = mastery["correct_predictions"].astype(float)
    else:
        raise KeyError(f"Could not locate posterior/state columns. Columns: {list(mastery.columns)}")

# 5) Keep latest attempt per (user, skill)
last_idx = mastery.groupby(["user_id","skill_name"])["seq_idx"].transform("max")
latest = mastery[mastery["seq_idx"] == last_idx].copy()

P_mastery = latest[["user_id","skill_name","p_mastery"]].reset_index(drop=True)
print("Mastery table shape:", P_mastery.shape)
display(P_mastery.head(10))


Unique skills: 119 | Rows: 24000
Mastery table shape: (14983, 3)


,user_id,skill_name,p_mastery
0,CSE240001,Algorithm Implementation Lab,0.93610
1,CSE240001,Arrays & Pointers,0.42075
2,CSE240001,Asymptotic Analysis,0.99993
3,CSE240001,BJT Characteristics,0.88224
4,CSE240001,CLT & Sampling,0.98180
5,CSE240001,CPU Organization & Registers,0.76022
6,CSE240001,Cellular Basics & Propagation,0.97710
7,CSE240001,Classes & Objects,0.86788
8,CSE240001,Complexity Experiments,0.93703
9,CSE240001,Concurrency & Recovery,0.89161


In [16]:
# =========================================================
# 7) Elective Recommender (Hybrid: content + difficulty [+ perf])
# =========================================================
from sentence_transformers import SentenceTransformer

course_topics = (topics.groupby("course_code")["topic_name"]
                        .apply(lambda x: "; ".join(map(str,x))).reset_index())
course_text = courses.merge(course_topics, on="course_code", how="left")
course_text["desc"] = course_text["course_title"].fillna("") + " | " + course_text["topic_name"].fillna("")

embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2", device=DEVICE)
course_vecs = embedder.encode(course_text["desc"].tolist(), convert_to_numpy=True, normalize_embeddings=True)

def student_profile_embedding(student_id: str):
    fb = feedback[feedback["student_id"]==student_id].copy()
    if fb.empty:
        return course_vecs.mean(axis=0)
    fb = fb.merge(course_text[["course_code","desc"]], on="course_code", how="left")
    fb["w_interest"] = fb["interest_rating"].clip(1,5)/5.0

    cur = reg[reg["student_id"]==student_id].copy()
    if cur.empty:
        fb["w_perf"] = 0.5
    else:
        pred = grade_model.predict(cur[FEATURES+CAT])
        cur = cur.assign(pred_final=pred)[["course_code","pred_final"]]
        fb = fb.merge(cur, on="course_code", how="left")
        fb["w_perf"] = fb["pred_final"].fillna(fb["pred_final"].mean() if not np.isnan(fb["pred_final"].mean()) else 60)/100.0

    fb["w"] = 0.6*fb["w_interest"] + 0.4*fb["w_perf"]
    vecs = embedder.encode(fb["desc"].fillna("").tolist(), convert_to_numpy=True, normalize_embeddings=True)
    w = fb["w"].values.reshape(-1,1)
    return (vecs*w).sum(axis=0)/max(1e-6,w.sum())

def recommend_courses(student_id: str, topk=5, exclude_current=True):
    prof = student_profile_embedding(student_id)
    sims = (course_vecs @ prof)
    df = course_text.copy()
    cur = reg[reg["student_id"]==student_id][["course_code"]].drop_duplicates()
    if exclude_current:
        df = df[~df["course_code"].isin(cur["course_code"])]
    fb = feedback[feedback["student_id"]==student_id][["course_code","perceived_difficulty"]]
    df = df.merge(fb, on="course_code", how="left")
    df["diff_penalty"] = 1 - (df["perceived_difficulty"].fillna(3)/5.0)
    df["score"] = 0.7*sims[df.index] + 0.3*df["diff_penalty"]
    return df.sort_values("score", ascending=False).head(topk)[
        ["course_code","course_title","semester_no","score"]
    ]

sample_student = students["student_id"].iloc[0]
recommend_courses(sample_student, topk=5)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,course_code,course_title,semester_no,score


In [17]:
# =========================================================
# 8) Learning Path Generator (Jinja2 template)
# =========================================================
from jinja2 import Template

TPL = Template("""
# Personalized Plan for {{ name }} — Sem {{ sem }}
**Attendance**: {{ attendance }}% {% if attendance < 85 %}(⚠ target ≥ 85%){% endif %}

**Predicted final**: {{ pred_final|round(1) }}% ({{ letter }})  
**Risk (after-mid)**: {{ (risk_prob*100)|round(1) }}%

## Focus Topics (next 2 weeks)
{% for t in weak_topics %}
- **{{ t.skill }}** — mastery {{ (t.p*100)|round(0) }}% → 2× 45min practice + 1× timed quiz
{% endfor %}

## Upcoming Topics
{% for u in upcoming %}
- **{{ u }}** — pre-read + short quiz before class
{% endfor %}

## Routine
- Mon/Wed/Fri: 1h weak-topic drills; Tue/Thu: 30m recap + spaced repetition
- Weekend: 1 mock (final-style) on weak topics

## Electives Next Term
{% for r in recs %}
- **{{ r.course_code }} — {{ r.course_title }}** (fit score {{ r.score|round(2) }})
{% endfor %}
""".strip())

# Helper inference encoders
AFTER_MID_CAT = [c for c in Xr_enc.columns if c not in Xr.columns]

def generate_learning_path(student_id: str, course_code: str):
    row = reg[(reg["student_id"]==student_id)&(reg["course_code"]==course_code)].tail(1)
    if row.empty:
        return f"# No data for {student_id} {course_code}"
    r = row.iloc[0]

    # predicted final
    pred = float(grade_model.predict(row[FEATURES+CAT])[0])
    # risk probability
    xr = row[[c for c in AFTER_MID if c in row.columns]].copy()
    xr_enc = pd.get_dummies(xr, columns=["course_type","course_code"], drop_first=True)
    for m in set(Xr_enc.columns)-set(xr_enc.columns):
        xr_enc[m]=0
    xr_enc = xr_enc[Xr_enc.columns]
    risk_prob = float(risk_model.predict_proba(xr_enc)[0,1])

    attendance = float(r["attendance_percentage"])
    letter = letter_from_pct(pred)

    # weak topics (BKT)
    pm = P_mastery[P_mastery["user_id"]==student_id].copy()
    c_topics = topics[topics["course_code"]==course_code]["topic_name"].unique().tolist()
    pm = pm[pm["skill_name"].isin(c_topics)]
    weak = pm.sort_values("p_mastery").head(3)
    weak_topics = [{"skill": s, "p": float(p)} for s,p in zip(weak["skill_name"], weak["p_mastery"])]

    seen = (assess[(assess["student_id"]==student_id)&(assess["course_code"]==course_code)]
              .merge(topics, on=["course_code","topic_id"], how="left")["topic_name"]
              .dropna().unique().tolist())
    all_topics = topics[topics["course_code"]==course_code]["topic_name"].tolist()
    upcoming = [t for t in all_topics if t not in seen][:3]

    recs = recommend_courses(student_id, topk=3).to_dict("records")
    name = students[students["student_id"]==student_id]["name"].iloc[0]
    sem  = int(r["semester_no"])

    return TPL.render(
        name=name, sem=sem, attendance=attendance,
        pred_final=pred, letter=letter, risk_prob=risk_prob,
        weak_topics=weak_topics, upcoming=upcoming, recs=recs
    )

# Demo (first student & first course)
demo_student = students["student_id"].iloc[0]
demo_course  = courses["course_code"].iloc[0]
print(generate_learning_path(demo_student, demo_course))


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

# Personalized Plan for Sami Rahman — Sem 3
**Attendance**: 91.14% 

**Predicted final**: 81.1% (A+)  
**Risk (after-mid)**: 0.0%

## Focus Topics (next 2 weeks)

- **Eigenvalues & Eigenvectors** — mastery 72.0% → 2× 45min practice + 1× timed quiz

- **Diagonalization** — mastery 82.0% → 2× 45min practice + 1× timed quiz

- **Gaussian Elimination & LU** — mastery 84.0% → 2× 45min practice + 1× timed quiz


## Upcoming Topics

- **Vector Spaces & Bases** — pre-read + short quiz before class

- **Gram-Schmidt & QR** — pre-read + short quiz before class

- **Determinants & Cofactors** — pre-read + short quiz before class


## Routine
- Mon/Wed/Fri: 1h weak-topic drills; Tue/Thu: 30m recap + spaced repetition
- Weekend: 1 mock (final-style) on weak topics

## Electives Next Term



In [18]:
# ==== Export current state & learning-path snapshot for ALL students/courses ====
# Output: current_state_<YYYY-MM-DD>.csv  (one row per student×course)
import pandas as pd
import numpy as np
from datetime import datetime

# --------- Helpers (use existing objects if present; otherwise build) ----------
def ensure_regression_table():
    global reg
    try:
        assert isinstance(reg, pd.DataFrame) and "final_percentage" in reg.columns
        return reg
    except Exception:
        # Rebuild regression table using functions from earlier cells
        def build_assessment_wide(df):
            piv = df.pivot_table(index=["student_id","course_code","semester_no"],
                                 columns="assessment_type", values="norm", aggfunc="mean").reset_index()
            for c in ["quiz1","quiz2","quiz3","assignment","mid","final"]:
                if c not in piv.columns: piv[c] = np.nan
            piv["CA_qa"] = piv[["quiz1","quiz2","quiz3","assignment"]].mean(axis=1, skipna=True).fillna(0)
            piv["Mid_norm"]   = piv["mid"].fillna(0)
            piv["Final_norm"] = piv["final"].fillna(0)
            return piv

        def add_course_meta(df):
            return df.merge(courses[["course_code","semester_no","course_type","credit_hours"]].drop_duplicates(),
                            on=["course_code","semester_no"], how="left")

        def join_engagement(piv):
            agg_e = engage.groupby(["student_id","course_code","semester_no"], as_index=False).agg({
                "attendance_percentage":"mean",
                "lms_logins":"sum","forum_posts":"sum","chatbot_interactions":"sum",
                "on_time_submission_rate":"mean"
            })
            out = piv.merge(agg_e, on=["student_id","course_code","semester_no"], how="left")
            return add_course_meta(out)

        def join_feedback(piv):
            fb = feedback.groupby(["student_id","course_code","semester_no"], as_index=False).agg({
                "interest_rating":"mean","perceived_difficulty":"mean"
            })
            return piv.merge(fb, on=["student_id","course_code","semester_no"], how="left")

        # recompute new-policy final %
        if "norm" not in assess.columns:
            assess["norm"] = (assess["raw_score"]/assess["max_score"]).clip(0,1)

        wide = build_assessment_wide(assess)
        att = engage.groupby(["student_id","course_code","semester_no"], as_index=False)["attendance_percentage"].mean()
        wide = wide.merge(att, on=["student_id","course_code","semester_no"], how="left")
        wide["Att"] = wide["attendance_percentage"].fillna(0)/100.0
        new_policy = wide[["student_id","course_code","semester_no","CA_qa","Mid_norm","Final_norm","Att"]].copy()
        new_policy["final_percentage"] = (0.10*new_policy["Att"] + 0.15*new_policy["CA_qa"]
                                          + 0.25*new_policy["Mid_norm"] + 0.50*new_policy["Final_norm"]) * 100.0

        X = build_assessment_wide(assess)
        X = join_engagement(X)
        X = join_feedback(X)
        df = X.merge(new_policy[["student_id","course_code","semester_no","final_percentage"]],
                     on=["student_id","course_code","semester_no"], how="left")

        # prior semester avg
        prior = (new_policy.assign(prior_sem=lambda d: d["semester_no"]-1)
                 .merge(new_policy[["student_id","semester_no","final_percentage"]]
                        .rename(columns={"semester_no":"prior_sem","final_percentage":"prior_pct"}),
                        on=["student_id","prior_sem"], how="left"))
        prior_gpa = (prior.groupby(["student_id","semester_no"], as_index=False)["prior_pct"]
                          .mean().rename(columns={"prior_pct":"prior_sem_avg"}))
        df = df.merge(prior_gpa, on=["student_id","semester_no"], how="left").fillna(0)
        return df

def ensure_letter_from_pct():
    try:
        letter_from_pct  # exists
        return letter_from_pct
    except NameError:
        def _letter_from_pct(p):
            return ("A+" if p>=80 else "A" if p>=75 else "A-" if p>=70 else
                    "B+" if p>=65 else "B" if p>=60 else "B-" if p>=55 else
                    "C+" if p>=50 else "C" if p>=45 else "D" if p>=40 else "F")
        return _letter_from_pct

def ensure_P_mastery():
    global P_mastery
    try:
        assert isinstance(P_mastery, pd.DataFrame) and {"user_id","skill_name","p_mastery"} <= set(P_mastery.columns)
        return P_mastery
    except Exception:
        # build a lightweight proxy: use correctness rate per topic as mastery if pyBKT not available
        tmp = (assess.merge(topics, on=["course_code","topic_id"], how="left")
                    .dropna(subset=["topic_name"]))
        tmp["correct"] = (tmp["raw_score"]/tmp["max_score"] >= 0.6).astype(int)
        P = (tmp.groupby(["student_id","topic_name"], as_index=False)["correct"]
                .mean().rename(columns={"student_id":"user_id","topic_name":"skill_name","correct":"p_mastery"}))
        return P

def ensure_models_ready():
    # returns callable predictors; if models missing, use simple heuristics
    def _grade_predict(rows):
        try:
            _ = grade_model
            return grade_model.predict(rows)
        except Exception:
            # heuristic = linear combo of mid/final/CA/attendance (same as label formula)
            return (0.10*rows["attendance_percentage"].fillna(0)/100.0 +
                    0.15*rows["CA_qa"].fillna(0) +
                    0.25*rows["Mid_norm"].fillna(0) +
                    0.50*rows["Final_norm"].fillna(0)) * 100.0

    def _risk_predict(rows_enc):
        try:
            _ = risk_model
            return risk_model.predict_proba(rows_enc)[:,1]
        except Exception:
            # heuristic: risk if current blended < 60%
            approx = (0.10*rows_enc.get("attendance_percentage",0)/100.0 +
                      0.15*rows_enc.get("CA_qa",0) +
                      0.25*rows_enc.get("Mid_norm",0)) * 100.0
            return (approx < 60).astype(float)
    return _grade_predict, _risk_predict

def ensure_recommender():
    try:
        recommend_courses  # exists
        return recommend_courses
    except NameError:
        # simple content-based using topic overlap count
        def _rec(student_id, topk=3, exclude_current=True):
            st_courses = reg[reg["student_id"]==student_id]["course_code"].unique().tolist()
            course_topics = topics.groupby("course_code")["topic_name"].apply(set)
            # pick topk by number of topics not yet taken
            scores = []
            taken_topics = set()
            for c in st_courses:
                taken_topics |= course_topics.get(c, set())
            for c in courses["course_code"].unique():
                if exclude_current and c in st_courses: 
                    continue
                s = len(course_topics.get(c,set()) - taken_topics)
                title = courses[courses["course_code"]==c]["course_title"].iloc[0]
                sem   = courses[courses["course_code"]==c]["semester_no"].iloc[0]
                scores.append({"course_code":c, "course_title":title, "semester_no":sem, "score":float(s)})
            return pd.DataFrame(scores).sort_values("score", ascending=False).head(topk)
        return _rec

# Build or reuse artifacts
reg = ensure_regression_table()
letter_from_pct = ensure_letter_from_pct()
P_mastery = ensure_P_mastery()
grade_predict, risk_predict = ensure_models_ready()
recommend_courses = ensure_recommender()

# Columns we’ll export
export_rows = []
run_date = datetime.utcnow().date().isoformat()

# Prepare encoder for risk (one-hot categories) if we trained one
try:
    Xr_enc  # from training cell
    risk_encoder_cols = list(Xr_enc.columns)
except Exception:
    risk_encoder_cols = None

# For each student × their courses
merged_names = students[["student_id","name"]]
reg2 = reg.merge(merged_names, on="student_id", how="left") \
          .merge(courses[["course_code","course_title","semester_no"]].drop_duplicates(),
                 on=["course_code","semester_no"], how="left")

# Ensure categorical dtypes match what grade model expects; if not, it will still work with heuristics
CAT = ["course_type","course_code"]
for c in CAT:
    if c in reg2.columns:
        reg2[c] = reg2[c].astype("category")

# AFTER_MID feature list (fallback)
AFTER_MID = [
    "quiz1","quiz2","quiz3","assignment","CA_qa","Mid_norm",
    "attendance_percentage","lms_logins","forum_posts","chatbot_interactions","on_time_submission_rate",
    "interest_rating","perceived_difficulty","credit_hours","course_type","course_code"
]
FEATURES = [
    "quiz1","quiz2","quiz3","assignment","CA_qa","Mid_norm","Final_norm",
    "attendance_percentage","lms_logins","forum_posts","chatbot_interactions","on_time_submission_rate",
    "interest_rating","perceived_difficulty","credit_hours","prior_sem_avg","course_type","course_code"
]

for _, r in reg2.iterrows():
    sid = r["student_id"]; cname = r.get("name","")
    ccode = r["course_code"]; ctitle = r["course_title"]; sem = int(r["semester_no"])
    attendance = float(r.get("attendance_percentage", 0.0))

    # Build 1-row frames for predictors
    row_df = pd.DataFrame([r[FEATURES]]).copy()
    # grade prediction
    try:
        pred_final = float(grade_predict(row_df)[0])
    except Exception:
        pred_final = float(r.get("final_percentage", 0.0))
    letter = letter_from_pct(pred_final)

    # risk prediction (encode if we have encoder columns)
    xr = pd.DataFrame([r[[c for c in AFTER_MID if c in r.index]]]).copy()
    if risk_encoder_cols is not None:
        xr_enc = pd.get_dummies(xr, columns=["course_type","course_code"], drop_first=True)
        # align columns
        for m in set(risk_encoder_cols) - set(xr_enc.columns):
            xr_enc[m] = 0
        xr_enc = xr_enc[risk_encoder_cols]
    else:
        # best-effort without encoder cache
        xr_enc = xr.select_dtypes(include=[np.number]).copy()
    try:
        risk_prob = float(risk_predict(xr_enc)[0])
    except Exception:
        risk_prob = 0.0

    # Weak topics (lowest mastery 3) for this course
    c_topics = topics[topics["course_code"]==ccode]["topic_name"].unique().tolist()
    pm = P_mastery[(P_mastery["user_id"].astype(str)==str(sid)) & (P_mastery["skill_name"].isin(c_topics))].copy()
    if not pm.empty:
        weak = pm.sort_values("p_mastery").head(3)
        weak_topics = "; ".join([f"{s} ({p:.2f})" for s,p in zip(weak["skill_name"], weak["p_mastery"])])
    else:
        weak_topics = ""

    # Upcoming topics (not seen by this student)
    seen = (assess[(assess["student_id"]==sid)&(assess["course_code"]==ccode)]
              .merge(topics, on=["course_code","topic_id"], how="left")["topic_name"]
              .dropna().unique().tolist())
    upcoming = [t for t in c_topics if t not in seen][:3]
    upcoming_topics = "; ".join(upcoming)

    # Elective recs (top-3)
    try:
        recs = recommend_courses(sid, topk=3)
        elective_recs = "; ".join([f"{row.course_code}:{row.course_title}({row.score:.2f})"
                                   for _,row in recs.iterrows()])
    except Exception:
        elective_recs = ""

    export_rows.append({
        "run_date": run_date,
        "student_id": sid,
        "name": cname,
        "semester_no": sem,
        "course_code": ccode,
        "course_title": ctitle,
        "attendance_percentage": attendance,
        "pred_final_pct": round(pred_final,2),
        "pred_letter": letter,
        "risk_prob_after_mid": round(risk_prob,3),
        "weak_topics": weak_topics,
        "upcoming_topics": upcoming_topics,
        "elective_recs_top3": elective_recs
    })

snapshot_df = pd.DataFrame(export_rows).sort_values(["student_id","semester_no","course_code"])
out_name = f"current_state_{run_date}.csv"
snapshot_df.to_csv(out_name, index=False)
print(f"Saved {out_name} with shape {snapshot_df.shape}")
display(snapshot_df.head(20))


Saved current_state_2025-10-07.csv with shape (4000, 13)


,run_date,student_id,name,semester_no,course_code,course_title,attendance_percentage,pred_final_pct,pred_letter,risk_prob_after_mid,weak_topics,upcoming_topics,elective_recs_top3
0,2025-10-07,CSE240001,Sami Rahman,3,CSE 4301,Object Oriented Programming,88.26000,73.81000,A-,0.00200,Exceptions (0.80); Templates & STL (0.81); Cla...,Encapsulation; Inheritance; Namespaces,
1,2025-10-07,CSE240001,Sami Rahman,3,CSE 4302,Object Oriented Programming Lab,83.52000,77.27000,A,0.00200,Inheritance/Polymorphism Lab (0.90); File I/O ...,OOP Lab Basics,
2,2025-10-07,CSE240001,Sami Rahman,3,CSE 4303,Data Structures,88.08000,74.64000,A-,0.00200,Arrays & Pointers (0.42); Linked Lists (0.75);...,Stacks & Queues; Graphs (BFS/DFS); Memory Mana...,
3,2025-10-07,CSE240001,Sami Rahman,3,CSE 4304,Data Structures Lab,93.50000,84.56000,A+,0.00100,Linked List Lab (0.85); Sorting & Hashing Lab ...,Stack/Queue Lab,
4,2025-10-07,CSE240001,Sami Rahman,3,CSE 4305,Computer Organization and Architecture,90.90000,75.69000,A,0.00100,DMA & Buses (0.74); CPU Organization & Registe...,ALU & Control Unit; RISC vs CISC,
5,2025-10-07,CSE240001,Sami Rahman,3,CSE 4307,Database Management Systems,93.03000,71.74000,A-,0.00100,Relational Model & Algebra (0.55); ER Modeling...,Normalization; SQL & Transactions; Indexing & ...,
6,2025-10-07,CSE240001,Sami Rahman,3,CSE 4308,Database Management Systems Lab,92.58000,80.47000,A+,0.00000,Transactions Lab (0.77); Indexing & Query Lab ...,,
13,2025-10-07,CSE240001,Sami Rahman,3,EEE 4383,Electronic Devices and Circuits,90.61000,71.04000,A-,0.01600,Rectifiers & Regulators (0.78); Operational Am...,Frequency Response & Noise; JFET & MOSFET; Pow...,
14,2025-10-07,CSE240001,Sami Rahman,3,EEE 4384,Electronic Devices and Circuits Lab,94.36000,74.25000,A-,0.00200,Rectifier/Regulator Labs (0.89); Diode/BJT Lab...,Op-amp Labs,
18,2025-10-07,CSE240001,Sami Rahman,3,Math 4341,Linear Algebra,91.14000,81.07000,A+,0.00000,Eigenvalues & Eigenvectors (0.72); Diagonaliza...,Vector Spaces & Bases; Gram-Schmidt & QR; Dete...,


In [20]:
# ==== Update after event + save fresh snapshot CSV (ALL students/courses) ====
import pandas as pd, numpy as np
from datetime import datetime
from pyBKT.models import Model

# ---- Utility: safe letter mapping (if not defined) ----
try:
    letter_from_pct
except NameError:
    def letter_from_pct(p):
        return ("A+" if p>=80 else "A" if p>=75 else "A-" if p>=70 else
                "B+" if p>=65 else "B" if p>=60 else "B-" if p>=55 else
                "C+" if p>=50 else "C" if p>=45 else "D" if p>=40 else "F")

# ---- Recompute BKT mastery (robust) ----
def compute_P_mastery(assess_df, topics_df):
    seq = assess_df.copy()
    seq["correct"] = (seq["raw_score"]/seq["max_score"] >= 0.6).astype(int)
    seq = (seq.merge(topics_df, on=["course_code","topic_id"], how="left")
             .dropna(subset=["topic_name"]))
    if seq.empty:
        return pd.DataFrame(columns=["user_id","skill_name","p_mastery"])
    seq_bkt = (seq.rename(columns={"student_id":"user_id","topic_name":"skill_name"})
                 [["user_id","skill_name","correct","timestamp"]]
                 .sort_values(["user_id","skill_name","timestamp"]).reset_index(drop=True))
    seq_bkt["user_id"]    = seq_bkt["user_id"].astype(str)
    seq_bkt["skill_name"] = seq_bkt["skill_name"].astype(str)
    seq_bkt["seq_idx"]    = seq_bkt.groupby(["user_id","skill_name"]).cumcount()
    # Fit BKT (you can batch this weekly if desired)
    bkt = Model(seed=42, num_fits=3)
    bkt.fit(data=seq_bkt[["user_id","skill_name","correct","timestamp"]])
    pred = bkt.predict(data=seq_bkt[["user_id","skill_name","correct","timestamp"]]).reset_index(drop=True)
    # align & extract mastery
    pred["user_id"]    = seq_bkt["user_id"].values
    pred["skill_name"] = seq_bkt["skill_name"].values
    pred["seq_idx"]    = seq_bkt["seq_idx"].values
    # posterior column variants
    pcol = None
    for c in pred.columns:
        if c.lower().startswith("posterior"):
            pcol = c; break
    if pcol is None and "state_predictions" in pred.columns:
        def to_known_prob(v):
            a = np.array(v, dtype=float).ravel()
            return float(a[1]) if a.size>=2 else float(a.max()) if a.size else np.nan
        pred["p_mastery"] = pred["state_predictions"].apply(to_known_prob)
    elif pcol is not None:
        pred["p_mastery"] = pred[pcol].astype(float)
    elif "correct_predictions" in pred.columns:
        pred["p_mastery"] = pred["correct_predictions"].astype(float)  # proxy fallback
    else:
        pred["p_mastery"] = np.nan
    last_idx = pred.groupby(["user_id","skill_name"])["seq_idx"].transform("max")
    latest = pred[pred["seq_idx"]==last_idx].copy()
    return latest[["user_id","skill_name","p_mastery"]].reset_index(drop=True)

# ---- Build regression table with your policy (10% Att, 15% QA, 25% Mid, 50% Final) ----
def build_regression_table(assess_df, engage_df, courses_df, feedback_df):
    df = assess_df.copy()
    if "norm" not in df.columns:
        df["norm"] = (df["raw_score"]/df["max_score"]).clip(0,1)
    piv = df.pivot_table(index=["student_id","course_code","semester_no"],
                         columns="assessment_type", values="norm", aggfunc="mean").reset_index()
    for c in ["quiz1","quiz2","quiz3","assignment","mid","final"]:
        if c not in piv.columns: piv[c] = np.nan
    piv["CA_qa"]    = piv[["quiz1","quiz2","quiz3","assignment"]].mean(axis=1, skipna=True).fillna(0)
    piv["Mid_norm"] = piv["mid"].fillna(0)
    piv["Final_norm"]=piv["final"].fillna(0)
    # attendance merge
    att = engage_df.groupby(["student_id","course_code","semester_no"], as_index=False)["attendance_percentage"].mean()
    X = piv.merge(att, on=["student_id","course_code","semester_no"], how="left")
    X["Att"] = X["attendance_percentage"].fillna(0)/100.0
    # new-policy final %
    X["final_percentage"] = (0.10*X["Att"] + 0.15*X["CA_qa"] + 0.25*X["Mid_norm"] + 0.50*X["Final_norm"])*100.0
    # course meta + feedback
    X = X.merge(courses_df[["course_code","semester_no","course_type","credit_hours"]].drop_duplicates(),
                on=["course_code","semester_no"], how="left")
    fb = feedback_df.groupby(["student_id","course_code","semester_no"], as_index=False).agg({
        "interest_rating":"mean","perceived_difficulty":"mean"
    })
    X = X.merge(fb, on=["student_id","course_code","semester_no"], how="left")
    # prior semester average
    prior = (X[["student_id","semester_no","final_percentage"]]
             .rename(columns={"semester_no":"prior_sem","final_percentage":"prior_pct"}))
    prior_gpa = (prior.groupby(["student_id","prior_sem"], as_index=False)["prior_pct"]
                      .mean().rename(columns={"prior_sem":"semester_no","prior_pct":"prior_sem_avg"}))
    X = X.merge(prior_gpa, on=["student_id","semester_no"], how="left").fillna(0)
    return X

# ---- Risk prediction helper (uses trained risk_model if available; else heuristic) ----
def predict_risk_rows(rows_df, encoder_cols=None):
    AFTER_MID = [
        "quiz1","quiz2","quiz3","assignment","CA_qa","Mid_norm",
        "attendance_percentage","lms_logins","forum_posts","chatbot_interactions","on_time_submission_rate",
        "interest_rating","perceived_difficulty","credit_hours","course_type","course_code"
    ]
    xr = rows_df[[c for c in AFTER_MID if c in rows_df.columns]].copy()
    try:
        # use trained logistic if present
        xr_enc = pd.get_dummies(xr, columns=["course_type","course_code"], drop_first=True)
        if encoder_cols is not None:
            for m in set(encoder_cols) - set(xr_enc.columns):
                xr_enc[m] = 0
            xr_enc = xr_enc[encoder_cols]
        return risk_model.predict_proba(xr_enc)[:,1]
    except Exception:
        # heuristic fallback (attendance+CA+Mid blended)
        approx = (0.10*xr.get("attendance_percentage",0)/100.0 +
                  0.15*xr.get("CA_qa",0) + 0.25*xr.get("Mid_norm",0))*100.0
        return (approx < 60).astype(float).values

# ---- Grade prediction helper (uses grade_model if available; else formula) ----
def predict_grade_rows(rows_df):
    FEATURES = [
        "quiz1","quiz2","quiz3","assignment","CA_qa","Mid_norm","Final_norm",
        "attendance_percentage","lms_logins","forum_posts","chatbot_interactions","on_time_submission_rate",
        "interest_rating","perceived_difficulty","credit_hours","prior_sem_avg","course_type","course_code"
    ]
    X = rows_df[[c for c in FEATURES if c in rows_df.columns]].copy()
    try:
        return grade_model.predict(X)
    except Exception:
        return (0.10*rows_df["attendance_percentage"].fillna(0)/100.0 +
                0.15*rows_df["CA_qa"].fillna(0) +
                0.25*rows_df["Mid_norm"].fillna(0) +
                0.50*rows_df["Final_norm"].fillna(0)) * 100.0

# ---- Export snapshot for ALL students/courses ----
def save_snapshot_csv(reg_df, P_mastery_df, filename=None):
    run_date = datetime.utcnow().date().isoformat()
    out_name = filename or f"current_state_{run_date}.csv"
    names = students[["student_id","name"]]
    merged = (reg_df.merge(names, on="student_id", how="left")
                    .merge(courses[["course_code","course_title","semester_no"]].drop_duplicates(),
                           on=["course_code","semester_no"], how="left"))
    # predictions
    merged["pred_final_pct"] = predict_grade_rows(merged).astype(float)
    merged["pred_letter"]    = merged["pred_final_pct"].apply(letter_from_pct)
    # risk (align to training encoder if available)
    try:
        risk_encoder_cols = list(Xr_enc.columns)  # from earlier training cell
    except Exception:
        risk_encoder_cols = None
    merged["risk_prob_after_mid"] = predict_risk_rows(merged, encoder_cols=risk_encoder_cols)
    # weak topics (join mastery)
    def weak_topics_for(row):
        c_topics = topics[topics["course_code"]==row["course_code"]]["topic_name"].unique().tolist()
        pm = P_mastery_df[(P_mastery_df["user_id"].astype(str)==str(row["student_id"])) &
                          (P_mastery_df["skill_name"].isin(c_topics))]
        if pm.empty: return ""
        w = pm.sort_values("p_mastery").head(3)
        return "; ".join([f"{s}({p:.2f})" for s,p in zip(w["skill_name"], w["p_mastery"])])
    merged["weak_topics"] = merged.apply(weak_topics_for, axis=1)
    # upcoming topics (not yet attempted)
    def upcoming_for(row):
        seen = (assess[(assess["student_id"]==row["student_id"]) & (assess["course_code"]==row["course_code"])]
                .merge(topics, on=["course_code","topic_id"], how="left")["topic_name"].dropna().unique().tolist())
        all_t = topics[topics["course_code"]==row["course_code"]]["topic_name"].tolist()
        up = [t for t in all_t if t not in seen][:3]
        return "; ".join(up)
    merged["upcoming_topics"] = merged.apply(upcoming_for, axis=1)
    # select/export
    out = merged[[
        "student_id","name","semester_no","course_code","course_title",
        "attendance_percentage","pred_final_pct","pred_letter",
        "risk_prob_after_mid","weak_topics","upcoming_topics"
    ]].copy()
    out.insert(0, "run_date", run_date)
    out.to_csv(out_name, index=False)
    print(f"Saved {out_name} with shape {out.shape}")
    return out_name, out

# ---- Main: call this when a NEW EVENT arrives ----
def on_new_event_and_snapshot(new_row: dict, refit_bkt: bool = True):
    """
    new_row: {
      'student_id', 'course_code', 'semester_no', 'assessment_type',
      'topic_id', 'raw_score', 'max_score', 'timestamp'
    }
    """
    global assess, reg, P_mastery
    # 1) append event
    assess = pd.concat([assess, pd.DataFrame([new_row])], ignore_index=True)
    if "norm" not in assess.columns:
        assess["norm"] = (assess["raw_score"]/assess["max_score"]).clip(0,1)
    else:
        assess["norm"] = (assess["raw_score"]/assess["max_score"]).clip(0,1)

    # 2) recompute mastery (full refit or skip for speed)
    if refit_bkt:
        try:
            P_mastery = compute_P_mastery(assess, topics)
        except Exception as e:
            print("BKT refit failed, falling back to correctness-rate proxy:", e)
            tmp = (assess.merge(topics, on=["course_code","topic_id"], how="left")
                        .dropna(subset=["topic_name"]))
            tmp["correct"] = (tmp["raw_score"]/tmp["max_score"] >= 0.6).astype(int)
            P_mastery = (tmp.groupby(["student_id","topic_name"], as_index=False)["correct"]
                           .mean().rename(columns={"student_id":"user_id","topic_name":"skill_name","correct":"p_mastery"}))

    # 3) rebuild regression table (features + new-policy label)
    reg = build_regression_table(assess, engage, courses, feedback)

    # 4) save a fresh snapshot CSV for all students/courses
    return save_snapshot_csv(reg, P_mastery)

# ------------------ Example usage ------------------
# demo_new = {
#     "student_id": students["student_id"].iloc[0],
#     "course_code": courses["course_code"].iloc[0],
#     "semester_no": int(courses["semester_no"].iloc[0]),
#     "assessment_type": "quiz2",
#     "topic_id": topics[topics["course_code"]==courses["course_code"].iloc[0]]["topic_id"].iloc[0],
#     "raw_score": 8, "max_score": 10, "timestamp": pd.Timestamp.utcnow()
# }
# fname, snapshot = on_new_event_and_snapshot(demo_new, refit_bkt=True)
# snapshot.head()
